In [1]:
import os
import fiona
import pandas as pd
from pyproj import Transformer
from shapely.geometry import shape
import math
import numpy as np
import glob

In [2]:
def find_shapefiles(directory):
    
    shapefiles = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.shp'):
                shapefiles.append(os.path.join(root, file))
    return shapefiles

In [3]:
main_directory = '//smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024'  
shapefiles = find_shapefiles(main_directory)
print("Shapefiles found:", shapefiles)

Shapefiles found: ['//smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024\\17Aug\\k38_veg_WGS.shp', '//smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024\\17Aug\\k38_edge_WGS.shp', '//smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024\\16Aug\\kamik_f38_v1.shp', '//smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024\\16Aug\\k38_diag1e_hd_ws_shifted_WGS.shp', '//smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024\\22Aug\\t12_h1_mon_WGS.shp', '//smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024\\10Aug\\shapefiles\\parsons_e02_v1.shp', '//smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024\\21Aug\\t12_v1_WGS.shp', '//smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024\\12Aug\\shapefiles\\pe02_camp3_WGS.shp', '//smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024\\12Aug\\sha

In [4]:
def reproject_and_extract_coordinates(shapefile_path, output_crs):
    
    coordinates = []
    with fiona.open(shapefile_path) as shp:
        input_crs = shp.crs['init']  
        transformer = Transformer.from_crs(input_crs, output_crs, always_xy=True)

        for feature in shp:
            geom = shape(feature['geometry'])
            if geom.geom_type == 'Point':
                x, y = transformer.transform(geom.x, geom.y)
                print(feature['properties'].keys())
                point_id = feature['properties'].get('Point Id', 'Unknown')
                z = feature['properties'].get('WGS84 Elli', 0)  
                coordinates.append((point_id, x, y, z))
            elif geom.geom_type in ['LineString', 'Polygon']:
                for coord in geom.coords:
                    x, y = transformer.transform(coord[0], coord[1])
                    z = coord[2] if len(coord) > 2 else 0  
                    coordinates.append((x, y, z))

    return coordinates

In [5]:
import pandas as pd
import os

def save_coordinates_to_csv(shapefile_path, coordinates):
   
    # Generate the CSV filename based on the shapefile name
    base_name = os.path.splitext(os.path.basename(shapefile_path))[0]
    dir_name = os.path.dirname(shapefile_path)
    csv_file = os.path.join(dir_name, f"{base_name}_coordinates.csv")
    
    # Create a DataFrame and save it to a CSV file
    df = pd.DataFrame(coordinates, columns=['ID', 'X', 'Y', 'Z (Ellipsoidal Height)'])
    df.to_csv(csv_file, index=False)
    
    print(f"Coordinates for {shapefile_path} saved to {csv_file}")


In [6]:
def save_coordinates_to_txt(shapefile_path, coordinates):
    
    # Generate the TXT filename based on the shapefile name
    base_name = os.path.splitext(os.path.basename(shapefile_path))[0]
    dir_name = os.path.dirname(shapefile_path)
    txt_file = os.path.join(dir_name, f"{base_name}_coordinates.txt")
    
    # Open the TXT file and write the data in a structured format
    with open(txt_file, 'w') as file:
        file.write("X\tY\tZ (Ellipsoidal Height)\n")  # Header for the TXT file
        
        for row in coordinates:
            x = row[1] if row[1] is not None else 0.0
            y = row[2] if row[2] is not None else 0.0
            z = row[3] if row[3] is not None else 0.0
            
            file.write(f"{x:.6f}\t{y:.6f}\t{z:.2f}\n")  # Write X, Y, Z
            
    print(f"Coordinates for {shapefile_path} saved to {txt_file}")



In [7]:
output_crs = 'EPSG:32608'  
shapefiles = find_shapefiles(main_directory)


for shapefile_path in shapefiles:
    print(f"Processing shapefile: {shapefile_path}")
    
    # Reproject and extract coordinates
    coordinates = reproject_and_extract_coordinates(shapefile_path, output_crs)
    
    # Save coordinates to a CSV file
    save_coordinates_to_csv(shapefile_path, coordinates)
    save_coordinates_to_txt(shapefile_path, coordinates)


Processing shapefile: //smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024\17Aug\k38_veg_WGS.shp
KeysView(<fiona.model.Properties object at 0x0000016158BF1DD0>)
KeysView(<fiona.model.Properties object at 0x0000016158BF3D10>)
KeysView(<fiona.model.Properties object at 0x0000016158BF3E90>)
KeysView(<fiona.model.Properties object at 0x0000016158BF8050>)
KeysView(<fiona.model.Properties object at 0x0000016158BF8190>)
KeysView(<fiona.model.Properties object at 0x0000016158BF8350>)
KeysView(<fiona.model.Properties object at 0x0000016158BF84D0>)
KeysView(<fiona.model.Properties object at 0x0000016158BF8610>)
KeysView(<fiona.model.Properties object at 0x0000016158BF8790>)
KeysView(<fiona.model.Properties object at 0x0000016158BF8850>)
KeysView(<fiona.model.Properties object at 0x0000016158BF8A10>)
KeysView(<fiona.model.Properties object at 0x0000016158BF8B90>)
KeysView(<fiona.model.Properties object at 0x0000016158BF8D10>)
KeysView(<fiona.model.Properties object at 0x0

function for calculating actual distances between the electrodes

In [8]:


# Function to calculate Euclidean distance between two points
def calculate_distance(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance

# Calculate distance from the first point to every other point
def calculate_distances_from_first(coordinates):
    first_point = coordinates[0]
    distances_from_first = [calculate_distance(first_point, coord) for coord in coordinates]
    return distances_from_first



In [12]:
coordinates = np.loadtxt(main_directory + '/11Aug/coords/p02_h1_ws_coords.txt', usecols=(1, 2)) # change whenever need
distances = calculate_distances_from_first(coordinates)
distances

[0.0,
 1.7728066135913214,
 4.007279306423108,
 5.970173415321251,
 7.9067210074541965,
 9.923463792228311,
 11.877631287227759,
 13.820118485447397,
 15.800310742413515,
 17.747181158727994,
 19.739061066887746,
 21.623164846693093,
 23.674932786392233,
 25.614210796932767,
 27.656563054638276,
 29.632288132781067,
 31.560421119532005,
 33.60163626964604,
 35.52482091001314,
 37.54495313924037,
 39.51756900758077,
 41.48408653970134,
 43.5060629366579,
 45.484695811160556,
 47.47979601716289,
 49.627349314534584,
 51.62173217761231,
 53.5725575390076,
 55.619777282727405,
 57.55304535764921,
 59.4932417356925,
 61.48918765433368,
 63.42700347038875,
 65.35577753326692,
 67.30130948043325,
 69.256232755013,
 71.22168781256784,
 73.12933403220512,
 75.11630113990003,
 77.04243783914292,
 79.0184709298053,
 81.01105363614778,
 82.97888520509184,
 84.84618823492693,
 86.7491882145103,
 88.7282229872857,
 90.72786867453131,
 92.64908350538623]